In [258]:
class SudokuBoard(object):
    def __init__(self):
        self.nboard = 3
        self.nsubboard = 3
        """
        self.board = [
            0, 0, 0,   8, 0, 4,  0, 0, 0,
            0, 6, 1,   0, 0, 0,  5, 0, 0,
            0, 4, 0,   1, 2, 0,  3, 0, 0,

            5, 0, 0,   0, 0, 0,  8, 2, 3,
            0, 8, 0,   0, 0, 0,  0, 6, 0,
            3, 0, 6,   0, 0, 0,  0, 0, 1,

            0, 0, 9,   0, 1, 6,  0, 4, 0,
            0, 0, 3,   0, 0, 0,  6, 7, 0,
            0, 0, 0,   7, 0, 2,  0, 0, 0,
        ]"""
        # Knuth's presentation:
        self.board = [
            0, 0, 3, 0, 1, 0, 0, 0, 0,
            4, 1, 5, 0, 0, 0, 0, 9, 0,
            2, 0, 6, 5, 0, 0, 3, 0, 0,
            
            5, 0, 0, 0, 8, 0, 0, 0, 9,
            0, 7, 0, 9, 0, 0, 0, 3, 2,
            0, 3, 8, 0, 0, 4, 0, 6, 0,
            
            0, 0, 0, 2, 6, 0, 4, 0, 3,
            0, 0, 0, 3, 0, 0, 0, 0, 8,
            3, 2, 0, 0, 0, 7, 9, 5, 0
        ]
        self.n = self.nboard * self.nsubboard
        self.constraints = []
        
    def printBoard(self):
        for rr in range(self.n):
            for cc in range(self.n):
                if cc % self.nsubboard == 0:
                    print('|', end=' ')
                print(self.board[self.n * rr + cc], end=' ')
            print('\n', end='')
            if rr % self.nsubboard == 2:
                print('\n', end='')
                
    def ij(self, item):
        # item = self.n * i + j
        return item // self.n, item % self.n,
        
    def sets(self, item, fnx):
        i, j = self.ij(item)
        sets = []
        for nn in range(self.n):
            sets += [self.board[fnx(i, j, nn)]] 
        if not (sets == list(set(sets))):
            assert AssertionError, "Inconsistent sets."
        return set(sets) - set([0])
    
    def row(self, item):
        def fnx(i, j, nn):
            return (self.n) * i + nn
        return self.sets(item, fnx)
            
    def column(self, item):
        def fnx(i, j, nn):
            return (self.n) * (nn) + j
        return self.sets(item, fnx)

    def subBoardYielder(self, item):
        i, j = self.ij(item)
        sbi, sbj = self.nsubboard * (i // self.nsubboard), self.nsubboard * (j // self.nsubboard)
        for ii in range(self.nsubboard):
            for jj in range(self.nsubboard):
                yield self.n * (sbi + ii) + sbj + jj

    def subBoard(self, item):
        sets = []
        for ii in self.subBoardYielder(item):
            sets += [self.board[ii]]
        if not (sets == list(set(sets))):
            assert AssertionError, "Inconsitent subboard"
        return set(sets)
    
    def printSubBoard(self, item):
        print([(ii, self.board[ii]) for ii in self.subBoardYielder(item)])

    def printSubBoardConstraints(self, item):
        print([(ii, self.constraints[ii]) for ii in self.subBoardYielder(item)])
            
    
    def subBoardConstraintSet(self, item):
        import collections
        sets = []
        for ii in self.subBoardYielder(item):
            sets += list(self.constraints[ii])
        count = [k for k, v in collections.Counter(sets).items() 
                   if v == 1 and k in self.constraints[item]]

        return None if count == [] else count[0]

def sudokuSolver(sb):
    constraints = []
    for item in range(len(sb.board)):
        iconstraints = set(idx for idx in range(1, sb.n+1)) if sb.board[item] == 0 else set([sb.board[item]])
        sb.constraints += [iconstraints]

    print('Original board:')
    sb.printBoard()
    while True:
        inc = 0
        for idx in range(2): # For iterative loops, since loop 1 initialized constraints 
                             # and loop 2 processes initialized constraints.
            for item in range(len(sb.board)):
                if not (sb.board[item] == 0):
                    continue
                    
                sb.constraints[item] -= sb.row(item) 
                sb.constraints[item] -= sb.column(item)
                sb.constraints[item] -= sb.subBoard(item)

                #sb.rowConstraintsSet(item) # TODO: don't think the current problem requires them.
                #sb.colConstraintsSet(item) # Whether any problem requires them is inconclusive.
                it = sb.subBoardConstraintSet(item)
                if it is not None:
                    sb.constraints[item] = set([it])
                    sb.board[item] = it
                    inc += 1
                    print('--> modifying', sb.ij(item), ' to ', it)
                    sb.printBoard()
                    continue
                    
                if len(sb.constraints[item]) == 1 and sb.board[item] == 0:
                    sb.board[item] = list(sb.constraints[item])[0]
                    inc += 1
                    print('--> modifying', sb.ij(item), ' to ', sb.board[item])
                    sb.printBoard()
                    continue

        if inc == 0:
            break
            
    print('Solution:')      
    sb.printBoard()
            
sudokuSolver(SudokuBoard())


Original board:
| 0 0 3 | 0 1 0 | 0 0 0 
| 4 1 5 | 0 0 0 | 0 9 0 
| 2 0 6 | 5 0 0 | 3 0 0 

| 5 0 0 | 0 8 0 | 0 0 9 
| 0 7 0 | 9 0 0 | 0 3 2 
| 0 3 8 | 0 0 4 | 0 6 0 

| 0 0 0 | 2 6 0 | 4 0 3 
| 0 0 0 | 3 0 0 | 0 0 8 
| 3 2 0 | 0 0 7 | 9 5 0 

--> modifying (4, 4)  to  5
| 0 0 3 | 0 1 0 | 0 0 0 
| 4 1 5 | 0 0 0 | 0 9 0 
| 2 0 6 | 5 0 0 | 3 0 0 

| 5 0 0 | 0 8 0 | 0 0 9 
| 0 7 0 | 9 5 0 | 0 3 2 
| 0 3 8 | 0 0 4 | 0 6 0 

| 0 0 0 | 2 6 0 | 4 0 3 
| 0 0 0 | 3 0 0 | 0 0 8 
| 3 2 0 | 0 0 7 | 9 5 0 

--> modifying (5, 0)  to  9
| 0 0 3 | 0 1 0 | 0 0 0 
| 4 1 5 | 0 0 0 | 0 9 0 
| 2 0 6 | 5 0 0 | 3 0 0 

| 5 0 0 | 0 8 0 | 0 0 9 
| 0 7 0 | 9 5 0 | 0 3 2 
| 9 3 8 | 0 0 4 | 0 6 0 

| 0 0 0 | 2 6 0 | 4 0 3 
| 0 0 0 | 3 0 0 | 0 0 8 
| 3 2 0 | 0 0 7 | 9 5 0 

--> modifying (8, 4)  to  4
| 0 0 3 | 0 1 0 | 0 0 0 
| 4 1 5 | 0 0 0 | 0 9 0 
| 2 0 6 | 5 0 0 | 3 0 0 

| 5 0 0 | 0 8 0 | 0 0 9 
| 0 7 0 | 9 5 0 | 0 3 2 
| 9 3 8 | 0 0 4 | 0 6 0 

| 0 0 0 | 2 6 0 | 4 0 3 
| 0 0 0 | 3 0 0 | 0 0 8 
| 3 2 0 | 0 4 

| 2 0 6 | 5 7 9 | 3 1 4 

| 5 6 2 | 1 8 3 | 7 4 9 
| 1 7 4 | 9 5 6 | 8 3 2 
| 9 3 8 | 7 2 4 | 5 6 1 

| 8 5 9 | 2 6 0 | 4 7 3 
| 6 4 7 | 3 9 0 | 1 2 8 
| 3 2 1 | 8 4 7 | 9 5 6 

--> modifying (6, 5)  to  1
| 7 0 3 | 4 1 0 | 6 8 5 
| 4 1 5 | 6 3 8 | 2 9 7 
| 2 0 6 | 5 7 9 | 3 1 4 

| 5 6 2 | 1 8 3 | 7 4 9 
| 1 7 4 | 9 5 6 | 8 3 2 
| 9 3 8 | 7 2 4 | 5 6 1 

| 8 5 9 | 2 6 1 | 4 7 3 
| 6 4 7 | 3 9 0 | 1 2 8 
| 3 2 1 | 8 4 7 | 9 5 6 

--> modifying (7, 5)  to  5
| 7 0 3 | 4 1 0 | 6 8 5 
| 4 1 5 | 6 3 8 | 2 9 7 
| 2 0 6 | 5 7 9 | 3 1 4 

| 5 6 2 | 1 8 3 | 7 4 9 
| 1 7 4 | 9 5 6 | 8 3 2 
| 9 3 8 | 7 2 4 | 5 6 1 

| 8 5 9 | 2 6 1 | 4 7 3 
| 6 4 7 | 3 9 5 | 1 2 8 
| 3 2 1 | 8 4 7 | 9 5 6 

--> modifying (0, 1)  to  9
| 7 9 3 | 4 1 0 | 6 8 5 
| 4 1 5 | 6 3 8 | 2 9 7 
| 2 0 6 | 5 7 9 | 3 1 4 

| 5 6 2 | 1 8 3 | 7 4 9 
| 1 7 4 | 9 5 6 | 8 3 2 
| 9 3 8 | 7 2 4 | 5 6 1 

| 8 5 9 | 2 6 1 | 4 7 3 
| 6 4 7 | 3 9 5 | 1 2 8 
| 3 2 1 | 8 4 7 | 9 5 6 

--> modifying (0, 5)  to  2
| 7 9 3 | 4 1 2 | 6 8 5 
|